# Model of reaction between plasma membrane and cytosolic species

We model a reaction between the cell interior and cell membrane in a 2D geometry:
- Cyto - 2D cell "volume"
- PM - 1D cell boundary

Model from Rangamani et al, 2013, Cell. A cytosolic species, A, reacts with a species on the PM, B, to form a new species on PM, X.


In [ ]:
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
img_A = mpimg.imread('axb-diagram.png')
plt.imshow(img_A)
plt.axis('off')

In [ ]:
import dolfin as d
import sympy as sym
import numpy as np
import pathlib

from smart import config, common, mesh, model, mesh_tools
from smart.units import unit
from smart.model_assembly import (
    Compartment,
    Parameter,
    Reaction,
    Species,
    SpeciesContainer,
    ParameterContainer,
    CompartmentContainer,
    ReactionContainer,
)
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

First, we define the various units for the inputs

In [ ]:
# Aliases - base units
um = unit.um
molecule = unit.molecule
sec = unit.sec
dimensionless = unit.dimensionless
# Aliases - units used in model
D_unit = um**2 / sec
surf_unit = molecule / um**2
flux_unit = molecule / (um * sec)
edge_unit = molecule / um

Next we generate the model.

In [ ]:
# =============================================================================================
# Compartments
# =============================================================================================
# name, topological dimensionality, length scale units, marker value
Cyto = Compartment("Cyto", 2, um, 1)
PM = Compartment("PM", 1, um, 3)
cc = CompartmentContainer()
cc.add([Cyto, PM])

# =============================================================================================
# Species
# =============================================================================================
# name, initial concentration, concentration units, diffusion, diffusion units, compartment
A = Species("A", 1.0, surf_unit, 1.0, D_unit, "Cyto")
X = Species("X", 1.0, edge_unit, 1.0, D_unit, "PM")
B = Species("B", 0.0, edge_unit, 1.0, D_unit, "PM")
sc = SpeciesContainer()
sc.add([A, X, B])

# =============================================================================================
# Parameters and Reactions
# =============================================================================================

# Reaction of A and X to make B (Cyto-PM reaction)
kon = Parameter("kon", 1.0, 1/(surf_unit*sec))
koff = Parameter("koff", 1.0, 1/sec)
r1 = Reaction("r1", ["A", "X"], ["B"], 
              param_map={"on": "kon", "off": "koff"}, 
              species_map={"A": "A", "X": "X", "B": "B"})

pc =ParameterContainer()
pc.add([kon, koff])
rc = ReactionContainer()
rc.add([r1])

We load the model generated above, and load in the mesh we will use in this example, iterating over 10 different values of cell radius, log-spaced between 1 and 10.

In [ ]:
# Create mesh
h_ellipse = 0.1
major_rad = 5.0
minor_rad = 1.0
inner_tag = 1
edge_tag = 3
ellipse_mesh, mf1, mf2 = mesh_tools.DemoEllipseMesh(major_rad, minor_rad, h_ellipse, inner_tag, edge_tag)

# Write mesh and meshfunctions to file
mesh_folder = pathlib.Path("ellipse_mesh")
mesh_folder.mkdir(exist_ok=True)
mesh_file = mesh_folder / "ellipse_mesh.h5"
hdf5 = d.HDF5File(ellipse_mesh.mpi_comm(), str(mesh_file.with_suffix(".h5")), "w")
hdf5.write(ellipse_mesh, "/mesh")
hdf5.write(mf2, "/mf2")
hdf5.write(mf1, "/mf1")
# For visualization of domains
d.File(str(mesh_file.with_stem(mesh_file.stem + "_mf2").with_suffix(".pvd"))) << mf2
d.File(str(mesh_file.with_stem(mesh_file.stem + "_mf1").with_suffix(".pvd"))) << mf1

parent_mesh = mesh.ParentMesh(
    mesh_filename=str(mesh_file),
    mesh_filetype="hdf5",
    name="parent_mesh",
)

In [ ]:
# Initialize model and solvers
configCur = config.Config()
modelCur = model.Model(pc, sc, cc, rc, configCur, parent_mesh)
configCur.solver.update(
    {
        "final_t": 1.0,
        "initial_dt": 0.01,
        "time_precision": 6,
        "use_snes": True,
        "print_assembly": False,
    }
)

modelCur.initialize(initialize_solver=False)
modelCur.initialize_discrete_variational_problem_and_solver()
# Write initial condition(s) to file
results = dict()
result_folder = pathlib.Path("resultsEllipse")
result_folder.mkdir(exist_ok=True)
for species_name, species in modelCur.sc.items:
    results[species_name] = d.XDMFFile(
        modelCur.mpi_comm_world, str(result_folder / f"{species_name}.xdmf")
    )
    results[species_name].parameters["flush_output"] = True
    results[species_name].write(modelCur.sc[species_name].u["u"], modelCur.t)

# Solve
while True:
    # Solve the system
    modelCur.monolithic_solve()
    # Save results for post processing
    for species_name, species in modelCur.sc.items:
        results[species_name].write(modelCur.sc[species_name].u["u"], modelCur.t)
    # End if we've passed the final time
    if modelCur.t >= modelCur.final_t:
        break

In [ ]:
%matplotlib inline
fig = plt.figure()
fig.set_size_inches(10, 10)
ax = fig.add_subplot(111, projection='3d')
d.plot(modelCur.sc["A"].u["u"])
ax.view_init(90, -90)
ax.set_box_aspect([major_rad,minor_rad,1])
ax.set_zticks([])